In [2]:
# read data

from mnist import MNIST
import numpy as np
mndata = MNIST("/Users/jigyayadav/Desktop/Codes/neuralnets253/HW1")
mndata.gz = True
images, labels = mndata.load_training() #Images is a list of 60000 images of 784 dimensions, Labels is a list of 60000 ints
imagesTest, labelsTest = mndata.load_testing()

In [38]:
images = np.array(images)
labels = np.array(labels)
imagesTest = np.array(imagesTest)
labelsTest = np.array(labelsTest)

images_train = images
images_test = imagesTest
labels_train = labels
labels_test = labelsTest
images_train = np.array(images_train)
labels_train = np.array(labels_train)
images_test = np.array(images_test)
labels_test = np.array(labels_test)

# Normalization
images_train = images_train*(1/127.5)
images_test = images_test*(1/127.5)
images_train = images_train-1.0
images_test = images_test-1.0

# Should this be appended after or before normalization
images_train = np.insert(images_train, 0, 1, axis=1)
images_test = np.insert(images_test, 0, 1, axis=1)

numFeatures = len(images[0, :])

# Divide between validation and training
from sklearn.model_selection import train_test_split
images_train, images_validation, labels_train, labels_validation = train_test_split(images_train, labels_train, test_size=0.10, random_state=42)

In [39]:
labelNums = 10

def labelsToVectForm(labels):
    vectForm = np.zeros((len(labels), labelNums))
    for i in range(len(labels)):
        vectForm[i][labels[i]] = 1
    return vectForm

t_train = labelsToVectForm(labels_train)
t_test = labelsToVectForm(labels_test)
t_validation = labelsToVectForm(labels_validation)

In [40]:
images_test.shape

(10000, 785)

In [ ]:
from sklearn.metrics import accuracy_score
import math

# Forward propagation

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmaxFunc(prod):
    prod = np.exp(prod)
    for i in range(len(prod)):
        rowSum = sum(prod[i, :])
        for j in range(len(prod[i, :])):
            prod[i][j] = prod[i][j]/rowSum    
    return prod

def getPredClass(y):
    predClass = np.argmax(y, axis=1)
    return predClass

def calculateOutput(x, W1, W2):
    A1 = x.dot(W1)
    Z1 = sigmoid(A1)
    X2 = np.insert(Z1, 0, 1, axis=1)
    A2 = X2.dot(W2)
    Y = softmaxFunc(A2)
    return Y

def calculateCost(x, t, W1, W2):
    Y = calculateOutput(x, W1, W2)
    cost = -(1/len(x))*(np.sum(np.multiply(t, np.log(Y))))
    return cost

def calculateCostOneInstance(x, t, W1, W2, n):
    Y = calculateOutput(x, W1, W2)
    cost = (-1/len(x))*(np.multiply(t, np.log(Y)))
    return np.sum(cost[n, :])

def calculateAccuracy(x, labels, W1, W2):
    Y = calculateOutput(x, W1, W2)
    predClass = getPredClass(Y)
    return accuracy_score(labels, predClass)

epsilon = 0.001
learningRate = 0.8

batch_size = 128
batches = images_train.shape[0]//batch_size

highestValidationAccuracy = float('-inf')
bestLearningRate = 0.1

for learningRate in [0.8, 0.1, 0.01, 0.001]:
    W1 = np.random.uniform(low=-1.0, high=1.0, size=(785, 64))
    W2 = np.random.uniform(low=-1.0, high=1.0, size=(65, 10))
    prevValidationError = float('inf')
    minValidationError = float('inf')
    W1MinError = np.zeros(shape=(785, 64))
    W2MinError = np.zeros(shape=(65, 10))
    cnt = 0
    epochMinWeight = 0
    stoppingIteration = 0
    for epoch in range(5000):
#         learningRate = learningRate/(1+epoch/10000)
        print("Epoch = ", epoch, " ", learningRate)
        for b in range(batches):
            x = images_train[b*batch_size:(b+1)*batch_size]
            t = t_train[b*batch_size:(b+1)*batch_size]
            A1 = x.dot(W1)
            Z1 = sigmoid(A1)

            X2 = np.insert(Z1, 0, 1, axis=1)
            A2 = X2.dot(W2)
            Y = np.exp(A2)/np.sum(np.exp(A2), axis=1, keepdims = True)

            # Backward propagation
            W2_nobias = np.delete(W2, (0), axis=0)
            summation_k = (Y-t).dot(W2_nobias.T)
            g_derivative = Z1*(1-Z1)
            delE_Wij = x.T.dot(np.multiply(summation_k, g_derivative))
            delE_Wij = delE_Wij

            # Check approximate derivative
    #         for n in range(0, 150, 30):
    #             print("For input to hidden layers")
    #             for i in range(0, 785, 100):
    #                 for j in range(0, 64, 20):
    #                     del_En_Wij = (g_derivative[n][j]*summation_k[n][j]*x[n][i])/len(x)
    #                     modifiedW1 = W1
    #                     modifiedW1[i][j] += epsilon
    #                     costFunction1 = calculateCostOneInstance(x, t, modifiedW1, W2, n)
    #                     modifiedW1[i][j] -= 2*epsilon
    #                     costFunction2 = calculateCostOneInstance(x, t, modifiedW1, W2, n)
    #                     approx_delEn = (costFunction1-costFunction2)/(2*epsilon)
    #                     diff = abs(del_En_Wij-approx_delEn)
    #     #                 print(del_En_Wij, " ", approx_delEn)
    #                     if (diff > epsilon*epsilon):
    #                         print("Difference between approximate gradient = ", diff, " ", abs(epsilon*epsilon-diff))

    #             print("For hidden to output layers")
    #             for j in range(0, 65, 20):
    #                 for k in range(0, 10, 3):
    #                     actualDerivative = X2[n][j]*(Y[n][k]-t[n][k])/len(x)
    #                     modifiedW2 = W2
    #                     modifiedW2[j][k] += epsilon
    #                     costFunction1 = calculateCostOneInstance(x, t, W1, modifiedW2, n)
    #                     modifiedW2[j][k] -= 2*epsilon
    #                     costFunction2 = calculateCostOneInstance(x, t, W1, modifiedW2, n)
    #                     approximateDerivative = (costFunction1-costFunction2)/(2*epsilon)
    #                     diff = abs(actualDerivative-approximateDerivative)
    #     #                 print(actualDerivative, " ", approximateDerivative)
    #                     if (diff > epsilon*epsilon):
    #                         print("Difference between approximate gradient = ", diff, " ", abs(epsilon*epsilon-diff))

            W2 = W2-(learningRate/len(x))*((X2.T.dot(Y-t)))
            W1 = W1-(learningRate/len(x))*(delE_Wij)

        # Calculate validation error
        currTrainingError = calculateCost(images_train, t_train, W1, W2)
        currvalidationError = calculateCost(images_validation, t_validation, W1, W2)
        currTrainingAccuracy = calculateAccuracy(images_train, labels_train, W1, W2)
        currValidationAccuracy = calculateAccuracy(images_validation, labels_validation, W1, W2)
        currTestAccuracy = calculateAccuracy(images_test, labels_test, W1, W2)
        if (currvalidationError < minValidationError):
            W1MinError = W1
            W2MinError = W2
            epochMinWeight = epoch
            minValidationError = currvalidationError

        # Early stopping
        if (currvalidationError > prevValidationError):
                cnt += 1
                if (cnt >= 3):
                    stoppingIteration = epoch
                    break
        else:
            cnt = 0
        prevValidationError = currvalidationError


        print("Current training error = ", currTrainingError)
        print("Current validation error = ", currvalidationError)
        print("Current training accuracy = ", currTrainingAccuracy)
        print("Current validation accuracy = ", currValidationAccuracy)
        print("Current test accuracy = ", currTestAccuracy)

    print("Iteration of min weights = ", epochMinWeight)
    print("Stopping iteration = ", stoppingIteration)
    currValidationAccuracy = calculateAccuracy(images_validation, labels_validation, W1MinError, W2MinError)
    if (currValidationAccuracy > highestValidationAccuracy):
        highestValidationAccuracy = currValidationAccuracy
        bestLearningRate = learningRate

print("Best learning rate = ", bestLearningRate)

Epoch =  0   0.8
Current training error =  0.39203969523392823
Current validation error =  0.3998110042904177
Current training accuracy =  0.8823148148148148
Current validation accuracy =  0.8771666666666667
Current test accuracy =  0.8869
Epoch =  1   0.8
Current training error =  0.31588680389754037
Current validation error =  0.3273095966279429
Current training accuracy =  0.904537037037037
Current validation accuracy =  0.9013333333333333
Current test accuracy =  0.9059
Epoch =  2   0.8
Current training error =  0.27878041463138675
Current validation error =  0.2932116865523221
Current training accuracy =  0.9178703703703703
Current validation accuracy =  0.9138333333333334
Current test accuracy =  0.9172
Epoch =  3   0.8
Current training error =  0.25172645069188887
Current validation error =  0.26853369466790533
Current training accuracy =  0.9259259259259259
Current validation accuracy =  0.9218333333333333
Current test accuracy =  0.9235
Epoch =  4   0.8
Current training error 

Current training error =  0.49900436461724235
Current validation error =  0.4997397348164474
Current training accuracy =  0.8496296296296296
Current validation accuracy =  0.8481666666666666
Current test accuracy =  0.8534
Epoch =  4   0.1
Current training error =  0.45811892372403273
Current validation error =  0.46091868659013263
Current training accuracy =  0.8629259259259259
Current validation accuracy =  0.8621666666666666
Current test accuracy =  0.8642
Epoch =  5   0.1
Current training error =  0.4289665321571613
Current validation error =  0.43359578689504286
Current training accuracy =  0.8711111111111111
Current validation accuracy =  0.8713333333333333
Current test accuracy =  0.8708
Epoch =  6   0.1
Current training error =  0.4064900582368333
Current validation error =  0.4124728841684774
Current training accuracy =  0.8781851851851852
Current validation accuracy =  0.8775
Current test accuracy =  0.8762
Epoch =  7   0.1
Current training error =  0.38827495878265234
Curren

Current training error =  0.20962969125267988
Current validation error =  0.24357079882611446
Current training accuracy =  0.9382777777777778
Current validation accuracy =  0.931
Current test accuracy =  0.9257
Epoch =  39   0.1
Current training error =  0.207132534879747
Current validation error =  0.2418838980646328
Current training accuracy =  0.9392222222222222
Current validation accuracy =  0.9313333333333333
Current test accuracy =  0.9261
Epoch =  40   0.1
Current training error =  0.20470693178194094
Current validation error =  0.24026146024453446
Current training accuracy =  0.9397222222222222
Current validation accuracy =  0.9313333333333333
Current test accuracy =  0.9262
Epoch =  41   0.1
Current training error =  0.20234876949347996
Current validation error =  0.23869863321457993
Current training accuracy =  0.9403148148148148
Current validation accuracy =  0.9318333333333333
Current test accuracy =  0.9269
Epoch =  42   0.1
Current training error =  0.2000537501908143
Cur

Current training error =  0.1501474004865522
Current validation error =  0.20476608866463406
Current training accuracy =  0.9562777777777778
Current validation accuracy =  0.9418333333333333
Current test accuracy =  0.9386
Epoch =  74   0.1
Current training error =  0.14900807147954537
Current validation error =  0.20396555779695408
Current training accuracy =  0.9565925925925925
Current validation accuracy =  0.9418333333333333
Current test accuracy =  0.9387
Epoch =  75   0.1
Current training error =  0.14788756537250813
Current validation error =  0.20317787487549124
Current training accuracy =  0.9568888888888889
Current validation accuracy =  0.942
Current test accuracy =  0.939
Epoch =  76   0.1
Current training error =  0.14678578435022643
Current validation error =  0.20240337115915796
Current training accuracy =  0.9572777777777778
Current validation accuracy =  0.9423333333333334
Current test accuracy =  0.9389
Epoch =  77   0.1
Current training error =  0.14570281751826536
C

Current training error =  0.11899168676031242
Current validation error =  0.18556550617897627
Current training accuracy =  0.9655555555555555
Current validation accuracy =  0.9468333333333333
Current test accuracy =  0.9424
Epoch =  109   0.1
Current training error =  0.11829592285904858
Current validation error =  0.18524367758130725
Current training accuracy =  0.9658518518518519
Current validation accuracy =  0.947
Current test accuracy =  0.9421
Epoch =  110   0.1
Current training error =  0.11760797199898192
Current validation error =  0.18493012668221645
Current training accuracy =  0.9662222222222222
Current validation accuracy =  0.947
Current test accuracy =  0.9422
Epoch =  111   0.1
Current training error =  0.11692755378009781
Current validation error =  0.184624861258369
Current training accuracy =  0.966462962962963
Current validation accuracy =  0.947
Current test accuracy =  0.9422
Epoch =  112   0.1
Current training error =  0.11625439242763545
Current validation error

...

Current training error =  0.13918627922577986
Current validation error =  0.20120414670203926
Current training accuracy =  0.959925925925926
Current validation accuracy =  0.9411666666666667
Current test accuracy =  0.9364
Epoch =  835   0.01
Current training error =  0.1390893696094654
Current validation error =  0.20116105481263488
Current training accuracy =  0.9599444444444445
Current validation accuracy =  0.9411666666666667
Current test accuracy =  0.9364
Epoch =  836   0.01
Current training error =  0.13899260648399578
Current validation error =  0.2011180683397476
Current training accuracy =  0.96
Current validation accuracy =  0.9411666666666667
Current test accuracy =  0.9364
Epoch =  837   0.01
Current training error =  0.13889598968212433
Current validation error =  0.20107518656423984
Current training accuracy =  0.9600185185185185
Current validation accuracy =  0.9411666666666667
Current test accuracy =  0.9364
Epoch =  838   0.01
Current training error =  0.1387995190329

Current training error =  0.13588032736098984
Current validation error =  0.19975433974709134
Current training accuracy =  0.9609629629629629
Current validation accuracy =  0.9408333333333333
Current test accuracy =  0.9374
Epoch =  870   0.01
Current training error =  0.13578842335553554
Current validation error =  0.19971456159084602
Current training accuracy =  0.9609814814814814
Current validation accuracy =  0.9408333333333333
Current test accuracy =  0.9374
Epoch =  871   0.01
Current training error =  0.135696658111416
Current validation error =  0.19967486885969302
Current training accuracy =  0.961
Current validation accuracy =  0.9408333333333333
Current test accuracy =  0.9374
Epoch =  872   0.01
Current training error =  0.13560503136605176
Current validation error =  0.1996352611932604
Current training accuracy =  0.9610185185185185
Current validation accuracy =  0.9408333333333333
Current test accuracy =  0.9375
Epoch =  873   0.01
Current training error =  0.135513542855

Current training error =  0.13283176341800695
Current validation error =  0.19844834410524573
Current training accuracy =  0.962037037037037
Current validation accuracy =  0.9413333333333334
Current test accuracy =  0.9385
Epoch =  904   0.01
Current training error =  0.13274441609966078
Current validation error =  0.19841135620139955
Current training accuracy =  0.9620740740740741
Current validation accuracy =  0.9413333333333334
Current test accuracy =  0.9385
Epoch =  905   0.01
Current training error =  0.13265719719309618
Current validation error =  0.1983744493314946
Current training accuracy =  0.9620925925925926
Current validation accuracy =  0.9411666666666667
Current test accuracy =  0.9385
Epoch =  906   0.01
Current training error =  0.13257010633671176
Current validation error =  0.19833762354451093
Current training accuracy =  0.9621296296296297
Current validation accuracy =  0.9411666666666667
Current test accuracy =  0.9386
Epoch =  907   0.01
Current training error =  

Current training error =  0.12984847445934877
Current validation error =  0.1972023329852995
Current training accuracy =  0.9629814814814814
Current validation accuracy =  0.9413333333333334
Current test accuracy =  0.9393
Epoch =  939   0.01
Current training error =  0.12976538589235373
Current validation error =  0.19716820930225804
Current training accuracy =  0.9629814814814814
Current validation accuracy =  0.9413333333333334
Current test accuracy =  0.9393
Epoch =  940   0.01
Current training error =  0.12968241136079267
Current validation error =  0.197134167633022
Current training accuracy =  0.9629814814814814
Current validation accuracy =  0.9413333333333334
Current test accuracy =  0.9393
Epoch =  941   0.01
Current training error =  0.12959955042010907
Current validation error =  0.19710020792611913
Current training accuracy =  0.963
Current validation accuracy =  0.9415
Current test accuracy =  0.9394
Epoch =  942   0.01
Current training error =  0.1295168026250529
Current

Current training error =  0.12700531420189215
Current validation error =  0.19605623302158648
Current training accuracy =  0.9638518518518518
Current validation accuracy =  0.9415
Current test accuracy =  0.9394
Epoch =  974   0.01


In [ ]:
# Final implementation with learning rate = 

numHiddenLayers = 64
numInputLayers = 785
numOutputLayers = 10

def gradientDescent(learningRate, T, shuffle, useMomentum, alpha, randomWeights):
    trainingAcc = []
    validationAcc = []
    testAcc = []
    trainingErr = []
    validationErr = []
    testErr = []
    if (useMomentum):
        v1 = np.zeros(shape=(numInputLayers, numHiddenLayers))
        v2 = np.zeros(shape=(numHiddenLayers+1, numOutputLayers))
    if (randomWeights):
        W1 = np.random.uniform(low=-1.0, high=1.0, size=(785, 64))
        W2 = np.random.uniform(low=-1.0, high=1.0, size=(65, 10))
    else:
        W1 = np.random.normal(loc=0.0, scale=1/sqrt(785), size=(785, 64))
        W2 = np.random.uniform(loc=0.0, scale=1/sqrt(65), size=(65, 10))
        
    prevValidationError = float('inf')
    minValidationError = float('inf')
    W1MinError = np.zeros(shape=(numInputLayers, numHiddenLayers))
    W2MinError = np.zeros(shape=(numHiddenLayers+1, numOutputLayers))
    cnt = 0
    epochMinWeight = 0
    stoppingIteration = 0
    for epoch in range(5000):
        print("Epoch = ", epoch)
        if (shuffle):
            images_train, t_train, labels_train = shuffle(images_train, t_train, labels_train)
        for b in range(batches):
            x = images_train[b*batch_size:(b+1)*batch_size]
            t = t_train[b*batch_size:(b+1)*batch_size]
            A1 = x.dot(W1)
            Z1 = sigmoid(A1)

            X2 = np.insert(Z1, 0, 1, axis=1)
            A2 = X2.dot(W2)
            Y = np.exp(A2)/np.sum(np.exp(A2), axis=1, keepdims = True)

            # Backward propagation
            W2_nobias = np.delete(W2, (0), axis=0)
            summation_k = (Y-t).dot(W2_nobias.T)
            g_derivative = Z1*(1-Z1)
            delE_Wij = x.T.dot(np.multiply(summation_k, g_derivative))
            
            delE_Wij = delE_Wij/len(x)
            delE_Wjk = (X2.T.dot(Y-t))/len(x)
            if (useMomentum):
                v1 = alpha*v1 - learningRate*delE_Wij
                v2 = alpha*v2 - learningRate*delE_Wjk
                delW1 = v1
                delW2 = v2
            else:
                delW1 = -1*learningRate*delE_Wij
                delW2 = -1*learningRate*delE_Wjk
            W2 = W2+delW2
            W1 = W1+delW1

        currTrainingError = calculateCost(images_train, t_train, W1, W2)
        currvalidationError = calculateCost(images_validation, t_validation, W1, W2)
        currTestError = calculateCost(images_test, t_test, W1, W2)
        currTrainingAccuracy = calculateAccuracy(images_train, labels_train, W1, W2)
        currValidationAccuracy = calculateAccuracy(images_validation, labels_validation, W1, W2)
        currTestAccuracy = calculateAccuracy(images_test, labels_test, W1, W2)
        if (currvalidationError < minValidationError):
            W1MinError = W1
            W2MinError = W2
            epochMinWeight = epoch
            minValidationError = currvalidationError

        # Early stopping
        if (currvalidationError > prevValidationError):
                cnt += 1
                if (cnt >= 3):
                    stoppingIteration = epoch
                    break
        else:
            cnt = 0
        prevValidationError = currvalidationError
        
        trainingErr.append(currTrainingError)
        validationErr.append(currvalidationError)
        testErr.append(currTestError)
        trainingAcc.append(currTrainingAccuracy)
        validationAcc.append(currValidationAccuracy)
        testAcc.append(currTestAccuracy)
        
    # Plot all errors and accuracies
        
    return [trainingErr, validationErr, testErr, trainingAcc, validationAcc, testAcc]

In [ ]:
po1 = [[1,2],[3,4],[9,8]]
po2 = [5,6,8]
po3 = [10,11,12]

po1, po2, po3 = shuffle(po1, po2, po3)